In [1]:
import requests
import zipfile
import io
import pandas as pd
import numpy as np

def download_statcan_table(table_id):

    # Step 1: Request metadata JSON
    meta_resp = requests.get(f"https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/{table_id}/en")
    meta_resp.raise_for_status()
    meta = meta_resp.json()

    # Step 2: Extract the download URL from metadata
    download_url = meta['object']

    # Step 3: Download and unzip the CSV file
    zip_resp = requests.get(download_url)
    zip_resp.raise_for_status()

    with zipfile.ZipFile(io.BytesIO(zip_resp.content)) as z:
        csv_filename = z.namelist()[0]
        with z.open(csv_filename) as f:
            df = pd.read_csv(f)
    return df

In [ ]:
# Batch retrieval code - defunct since running this because of RAM constraints, but can be adjusted as needed if useful
# table_ids = ["14100443","14100464"]
# dataframes = {}

# for table_id in table_ids:
#     for col, parameters in zip(cols, params):
#         df = download_statcan_table(table_id, col=col, parameters=parameters)
#         key = f"{table_id}_{col}"  # unique key per filter
#         dataframes[key] = df

In [ ]:
#querying the following dataset for a specific set of values: https://www150.statcan.gc.ca/t1/tbl1/en/cv.action?pid=1410028703
# 1) Baseline labour force characteristics / imbalanaces data for provinces by month
lfc_df = download_statcan_table('14100287')

lfs = (
    lfc_df.query(
        "GEO != 'Canada' and "
        "REF_DATE >= '2015-05' and REF_DATE <= '2025-07' and "
        "`Labour force characteristics` in ['Participation rate', 'Unemployment', 'Unemployment rate', 'Employment rate'] and "
        "Gender == 'Total - Gender' and "
        "Statistics == 'Estimate' and "
        "`Age group` == '15 years and over' and "
        "`Data type` == 'Seasonally adjusted'"
    )
    .assign(
        VALUE=lambda x: np.where(
            x['Labour force characteristics'] == 'Unemployment',  #necessary for this series alone as the original data is scaled by thousands
            x['VALUE'] * 1000,
            x['VALUE']
        )
    )
    .loc[:, ['GEO','Labour force characteristics', 'REF_DATE', 'VALUE']]
    .reset_index(drop=True)
)

lfs_pivoted = lfs.pivot_table(
    index=['GEO', 'REF_DATE'],
    columns='Labour force characteristics',
    values='VALUE'
).reset_index()

/tmp/ipython-input-4227337276.py:24: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


In [ ]:
#UE/employment rate/participation Rates by ER, annual (for map) https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410046401
lfc_er_df = download_statcan_table('14100464')

canada_pt = [
    "Canada",
    "Alberta",
    "British Columbia",
    "Manitoba",
    "New Brunswick",
    "Newfoundland and Labrador",
    "Nova Scotia",
    "Ontario",
    "Quebec",
    "Saskatchewan"
]

lfs_er = (
    lfc_er_df.query(
        "GEO not in @canada_pt and " # removing all provinces unless it is its own economic region
        "REF_DATE >= 2015 and REF_DATE <= 2024 and "
        "`Labour force characteristics` in ['Participation rate', 'Unemployment', 'Unemployment rate', 'Employment rate'] "
    )
    .assign(
        VALUE=lambda x: np.where(
            x['Labour force characteristics'] == 'Unemployment',  #necessary for this series alone as the original data is scaled by thousands
            x['VALUE'] * 1000,
            x['VALUE']
        )
    )
    .loc[:, ['GEO','Labour force characteristics', 'REF_DATE', 'VALUE']]
    .reset_index(drop=True)
)

lfs_er_pivoted = lfs_er.pivot_table(
    index=['GEO', 'REF_DATE'],
    columns='Labour force characteristics',
    values='VALUE'
).reset_index()

# necessary renamings due to ER name misalignments between JVWS and LFS :/
lfs_er_pivoted['GEO'] = lfs_er_pivoted['GEO'].replace('Parklands and Northern, Manitoba', 'Parklands and North, Manitoba')
lfs_er_pivoted['GEO'] = lfs_er_pivoted['GEO'].replace('South Coast-Burin Peninsula and Notre Dame-Central-Bonavista Bay, Newfoundland and Labrador', 'South Coast-Burin Peninsula and Notre Dame-Central Bonavista Bay, Newfoundland and Labrador')

# to add: extract province name as well to use as filter in dashboard

In [ ]:
working_age_population = (
    lfc_df.query(
        "GEO != 'Canada' and "
        "REF_DATE >= '2015-05' and REF_DATE <= '2025-07' and "
        "`Labour force characteristics` in ['Population'] and "
        "Gender == 'Total - Gender' and "
        "Statistics == 'Estimate' and "
        "UOM == 'Persons in thousands' and "
        "`Age group` in ['15 to 24 years', '25 to 54 years', '15 years and over'] and " #'55 to 64 years' no available data, separate calc
        "`Data type` == 'Seasonally adjusted'"
    )
    .assign(VALUE=lambda x: x['VALUE'] * 1000)
    .loc[:, ['GEO', 'Age group' ,'REF_DATE', 'VALUE']]
    .reset_index(drop=True)
)

population_pct = (
    working_age_population
    .merge(
        working_age_population.query("`Age group` == '15 years and over'")
        .rename(columns={'VALUE': 'total_over15'})[['GEO', 'REF_DATE', 'total_over15']],
        on=['GEO', 'REF_DATE'],
        how='left'
    )
    .assign(
        **{'pop_pct': lambda x: (x['VALUE'] / x['total_over15'] * 100).round(2)}
    )
    .drop(columns='total_over15')
)

population_pct['pop_pct'] = population_pct['pop_pct'].round(2)

population_pct = population_pct[['GEO', 'Age group','REF_DATE', 'VALUE' ,'pop_pct']]

## investigate for territories, find 55+ group?

In [ ]:
#territories LFS data retrieval from https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410029201

lfc_t_df = download_statcan_table('14100292')

lfs_t = (
    lfc_t_df.query(
        "GEO != 'Canada' and "
        "REF_DATE >= '2015-05' and REF_DATE <= '2025-07' and "
        "`Labour force characteristics` in ['Participation rate', 'Unemployment', 'Unemployment rate', 'Employment rate'] and "
        "Gender == 'Total - Gender' and "
        "Statistics == 'Estimate' and "
        "`Age group` == '15 years and over' and "
        "`Data type` == 'Unadjusted'"
    )
    .assign(
        VALUE=lambda x: np.where(
            x['Labour force characteristics'] == 'Unemployment',  #necessary for this series alone as the original data is scaled by thousands
            x['VALUE'] * 1000,
            x['VALUE']
        )
    )
    .loc[:, ['GEO','Labour force characteristics', 'REF_DATE', 'VALUE']]
    .reset_index(drop=True)
)

lfs_t_pivoted = lfs_t.pivot_table(
    index=['GEO', 'REF_DATE'],
    columns='Labour force characteristics',
    values='VALUE'
).reset_index()

/tmp/ipython-input-4227337276.py:24: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


In [ ]:
# same thing for vacancies: https://www150.statcan.gc.ca/t1/tbl1/en/cv.action?pid=1410043201

# Vacancies, adjusted for seasonality, by province, monthly

jv_df = download_statcan_table('14100432')

vacancies = (
    jv_df.query(
        "GEO != ['Canada', 'Yukon', 'Northwest Territories', 'Nunavut'] and " #filtering out territories for now
        "REF_DATE >= '2015-05' and REF_DATE <= '2025-07' and "
        "Statistics in ['Job vacancies','Job vacancy rate'] "
    )
    [['GEO', 'Statistics', 'REF_DATE', 'VALUE']]
    .reset_index(drop=True)
)

jvws_pivoted = vacancies.pivot_table(
    index=['GEO', 'REF_DATE'],
    columns='Statistics',
    values='VALUE'
).reset_index()

In [ ]:
# separate query for territories and calculate 3-month averages. Seasonality adjusted

t_vacancies = (
    jv_df.query(
        "GEO in ['Yukon', 'Northwest Territories', 'Nunavut'] and "
        "REF_DATE >= '2015-05' and REF_DATE <= '2025-07' and "
        "Statistics in ['Job vacancies','Job vacancy rate'] "
    )
    [['GEO', 'Statistics', 'REF_DATE', 'VALUE']]
    .reset_index(drop=True)
)

t_vacancies_pivot = t_vacancies.pivot_table(
    index=['GEO', 'REF_DATE'],
    columns='Statistics',
    values='VALUE'
).reset_index()

t_vacancies_pivot['VAC_3mm'] = (
    t_vacancies_pivot
    .groupby('GEO')['Job vacancies']
    .transform(lambda x: x.rolling(window=3, min_periods=3).apply(
        lambda w: w.mean() if not w.isna().any() else pd.NA
    ).round(2))
)

In [ ]:
# archived job vacancies data (quarterly) for ERs, non- seasonally adjusted https://www150.statcan.gc.ca/t1/tbl1/en/cv.action?pid=1410032501
jvws_er_df = download_statcan_table('14100325')

jvws_er = (
    jvws_er_df.query(
        "GEO not in @canada_pt and " # canada_pt defined earlier
        "REF_DATE >= '2015-01' and REF_DATE <= '2022-10' and "
        "Statistics == 'Job vacancies'"
    )
    .loc[:, ['GEO','Statistics', 'REF_DATE', 'VALUE']]
    .reset_index(drop=True)
)

jvws_er_pivoted = jvws_er.pivot_table(
    index=['GEO', 'REF_DATE'],
    columns='Statistics',
    values='VALUE'
).reset_index()

jvws_er_pivoted['REF_DATE'] = jvws_er_pivoted['REF_DATE'].str[:4]  # get 'YYYY' from 'YYYY-MM'

jvws_er_annual = (
    jvws_er_pivoted
    .groupby(['REF_DATE', 'GEO'], as_index=False)['Job vacancies']
    .apply(lambda x: x.sum() if (x.notna().sum() == 4) else np.nan)
)

jvws_er_annual['REF_DATE'] = jvws_er_annual['REF_DATE'].astype(int)

In [ ]:
# tab for wages offered https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410044301

w_offered_url = "https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/14100443/en"
w_offered_resp = requests.get(w_offered_url)

w_offered_meta = w_offered_resp.json()
w_offered_download_url = w_offered_meta['object']

w_offered_zip_resp = requests.get(w_offered_download_url)

with zipfile.ZipFile(io.BytesIO(w_offered_zip_resp.content)) as z:
    w_offered_csv_filename = z.namelist()[0]
    with z.open(w_offered_csv_filename) as f:
        chunks = []
        for chunk in pd.read_csv(f, chunksize=100000):
            filtered = chunk[chunk['Statistics'] == 'Average offered hourly wage'] #custom retrieval to filter for stat so it doesn't take a million years to run
            chunks.append(filtered)
        w_offered_df = pd.concat(chunks, ignore_index=True)

wages_offered = (
    w_offered_df.query(
        "GEO != ['Canada'] and "
        "REF_DATE >= '2015-04' and REF_DATE <= '2025-04' and "
        "`National Occupational Classification` == 'Total, all occupations' and "
        "`Job vacancy characteristics` in ['Type of work, all types','Full-time','Part-time']"
    )
    [['GEO', 'REF_DATE', 'VALUE','Job vacancy characteristics']]
    .reset_index(drop=True)
)

wages_offered['Year'] = wages_offered['REF_DATE'].str[:4]  # get 'YYYY' from 'YYYY-MM'

annual_wages_offered = (
    wages_offered
    .groupby(['Year', 'GEO', 'Job vacancy characteristics'], as_index=False)['VALUE']
    .mean()
    .rename(columns={'VALUE': 'offered_wage'})
    .assign(**{'offered_wage': lambda x: x['offered_wage'].round(2)})
)

#investigate: wages by 2-digit or 3-digit noc, industries available?

In [ ]:
# wages earned, annual https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410041701

w_earned_url = "https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/14100417/en"
w_earned_resp = requests.get(w_earned_url)

w_earned_meta = w_earned_resp.json()
w_earned_download_url = w_earned_meta['object']

w_earned_zip_resp = requests.get(w_earned_download_url)

with zipfile.ZipFile(io.BytesIO(w_earned_zip_resp.content)) as z:
    w_earned_csv_filename = z.namelist()[0]
    with z.open(w_earned_csv_filename) as f:
        chunks = []
        for chunk in pd.read_csv(f, chunksize=100000):
            filtered = chunk[chunk['Wages'] == 'Average hourly wage rate']
            chunks.append(filtered)
        w_earned_df = pd.concat(chunks, ignore_index=True)

wages_earned = (
    w_earned_df.query(
        "GEO != ['Canada'] and "
        "REF_DATE>= 2015 and REF_DATE <= 2025 and "
        "`National Occupational Classification (NOC)` == 'Total employees, all occupations [00-95]' and "
        "`Type of work` in ['Both full- and part-time employees', 'Full-time employees', 'Part-time employees'] and "
        "Gender == 'Total - Gender' and "
        "`Age group` == '15 years and over' "
    )
    [['GEO', 'REF_DATE', 'Type of work', 'VALUE']]
    .reset_index(drop=True)
)

#investigate: wages by 2-digit or 3-digit noc, industries available?

In [ ]:
# wages data merged
annual_wages_offered['REF_DATE'] = annual_wages_offered['Year'].astype(int)
annual_wages_offered = annual_wages_offered.rename(columns={
    'Job vacancy characteristics': 'Type of work'
})

annual_wages_offered['Type of work'] = annual_wages_offered['Type of work'].replace({
    'Type of work, all types': 'Both full- and part-time employees',
    'Full-time': 'Full-time employees',
    'Part-time': 'Part-time employees',
})

wages_earned = wages_earned.rename(columns={
    'VALUE': 'earned_wage'
})

wages = pd.merge(
    annual_wages_offered,
    wages_earned,
    on=['GEO', 'REF_DATE', 'Type of work'],
    how='left'
)

In [27]:
# lf by edu attainment bar graph: Labour force characteristics by educational degree, annual https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410011801

lfs_edu_df = download_statcan_table('14100118')

lfs_edu = (
    lfs_edu_df.query(
    "GEO != ['Canada'] and "
    "REF_DATE>= 2015 and REF_DATE <= 2025 and "
    "`Labour force characteristics` == 'Labour force' and "
    "Gender == 'Total - Gender' and "
    "`Educational degree` in ["
    "\"Total, all education levels\", "
    "\"Above bachelor's degree\", \"Bachelor's degree\", "
    "\"Postsecondary certificate or diploma\", "
    "\"High school graduate, some post-secondary\", "
    "\"High school graduate\", \"No degree, certificate or diploma\""
    "] and "
    "`Age group` == '15 years and over'"
)
.assign(VALUE=lambda x: x["VALUE"] * 1000)
.loc[:, ['GEO','Educational degree', 'REF_DATE', 'VALUE']]
    .reset_index(drop=True)
)

In [29]:
# employment by occupation? (Table 14-10-0416-01 Labour force characteristics by occupation, annual)

emp_noc_df = download_statcan_table('14100416')
emp_noc = (
    emp_noc_df.query(
    "`GEO` != 'Canada' and "
    "REF_DATE >= 2015 and REF_DATE <= 2025 and "
    "`Labour force characteristics` == 'Proportion of employment' and "
    "Gender == 'Total - Gender'  "
)
.assign(VALUE=lambda x: x["VALUE"] * 1000)
.loc[:, ['GEO','National Occupational Classification (NOC)', 'REF_DATE', 'VALUE']]
    .reset_index(drop=True)
)

# clean noc (indicator for 2 or 3 digit, separate code and name)

In [ ]:
# vacancies by occ Table 14-10-0443-01 Job vacancies, proportion of job vacancies and average offered hourly wage by occupation and selected characteristics, quarterly, unadjusted for seasonality

vacancies_noc_df = download_statcan_table('14100443')

In [5]:
# LR UE and Vacancies https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410005701 for UE (annual)

lr_ue_df = download_statcan_table('14100057')

lr_ue = (
    lr_ue_df.query(
        "GEO != ['Canada'] and "
        "REF_DATE>= 2015 and REF_DATE <= 2025 and "
        "`Duration of unemployment` == '53 weeks or more' and "
        "Gender == 'Total - Gender' and "
        "`Age group` == '15 years and over' "
    )
.assign(VALUE=lambda x: x["VALUE"] * 1000)
.loc[:, ['GEO', 'REF_DATE', 'VALUE']]
    .reset_index(drop=True)
)

In [6]:
# long run vacancies (quarterly) - https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410044301
lr_vacancies_url = "https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/14100443/en"
lr_vacancies_resp = requests.get(lr_vacancies_url)

lr_vacancies_meta = lr_vacancies_resp.json()
lr_vacancies_download_url = lr_vacancies_meta['object']

lr_vacancies_zip_resp = requests.get(lr_vacancies_download_url)

with zipfile.ZipFile(io.BytesIO(lr_vacancies_zip_resp.content)) as z:
    lr_vacancies_csv_filename = z.namelist()[0]
    with z.open(lr_vacancies_csv_filename) as f:
        chunks = []
        for chunk in pd.read_csv(f, chunksize=100000):
            filtered = chunk[chunk['Job vacancy characteristics'] == '90 days or more']
            chunks.append(filtered)
        lr_vacancies_df = pd.concat(chunks, ignore_index=True)

lr_vacancies = (
    lr_vacancies_df.query(
        "GEO != ['Canada'] and "
        "REF_DATE>= '2015-01' and REF_DATE <= '2025-04' and "
        # "`Job vacancy characteristics` == '90 days or more' and"
        "`National Occupational Classification` == 'Total, all occupations' and "
        "Statistics  == 'Job vacancies' "
    )
.loc[:, ['GEO', 'REF_DATE', 'VALUE']]
    .reset_index(drop=True)
)

,GEO,REF_DATE,VALUE
0,Newfoundland and Labrador,2015-01,NaN
1,Prince Edward Island,2015-01,NaN
2,Nova Scotia,2015-01,NaN
3,New Brunswick,2015-01,NaN
4,Quebec,2015-01,6185.0
...,...,...,...
515,Alberta,2025-04,15055.0
516,British Columbia,2025-04,24620.0
517,Yukon,2025-04,300.0
518,Northwest Territories,2025-04,360.0


In [ ]:
# hours worked https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410004301 for annual and https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410004201 for month/province (none for territories?)


In [ ]:
# to add:
# calculate annual ue/v ratio to contrast against wage differences or earned wages?

In [ ]:
# merges and tweaks

lfs_jvws = pd.merge(lfs_pivoted, jvws_pivoted, on=["GEO", "REF_DATE"], how="inner")
t_lfs_jvws = pd.merge(lfs_t_pivoted, t_vacancies_pivot, on=["GEO", "REF_DATE"], how="inner")
er_lfs_jvws = pd.merge(lfs_er_pivoted, jvws_er_annual, on=["GEO", "REF_DATE"], how="outer")
er_lfs_jvws.loc[~er_lfs_jvws['GEO'].str.startswith('Northeast'), 'GEO'] = (
    er_lfs_jvws['GEO'].str.replace(r',.*', '', regex=True)
)

In [ ]:
# exports
lfs_jvws.to_excel("lfs_jvws.xlsx", index=False)
population_pct.to_excel("population_pct.xlsx", index = False)
er_lfs_jvws.to_excel("er_lfs_jvws.xlsx", index=False)
wages.to_excel("wages.xlsx",index = False)
t_lfs_jvws("territories_lfs_jvws.xlsx", index=False)
lr_vacancies.to_excel("lr_vacancies.xlsx",index=False)
lr_ue.to_excel("lr_ue.xlsx", index=False)
lfs_edu.to_excel("lfs_edu.xlsx",index=False)